![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# LLM Session Memory

Large Language Models are inherently stateless and have no knowledge of previous interactions with a user, or even of previous parts of the current conversation. While this may not be noticeable when asking simple questions, it becomes a hinderance when engaging in long running conversations that rely on conversational context.

The solution to this problem is to append the previous conversation history to each subsequent call to the LLM.

This notebook will show how to use Redis to structure and store and retrieve this conversational session memory.

## Let's Begin!
<a href="https://colab.research.google.com/github/redis-developer/redis-ai-resources/blob/main/python-recipes/session-memory/session_memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Environment setup

## set cohere api key

In [1]:
import os
import getpass


if "COHERE_API_KEY" not in os.environ:
    os.environ["COHERE_API_KEY"] = getpass.getpass("COHERE_API_KEY")

# P6apFmSFsvRxrxoU7Nd2gtD701m6teUNHirY7xiI

## Run local redis (for colab)
Use the shell script below to download, extract, and install [Redis Stack](https://redis.io/docs/getting-started/install-stack/) directly from the Redis package archive.

In [ ]:
# NBVAL_SKIP
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

## For Alternative Environments
There are many ways to get the necessary redis-stack instance running
1. On cloud, deploy a [FREE instance of Redis in the cloud](https://redis.com/try-free/). Or, if you have your
own version of Redis Enterprise running, that works too!
2. Per OS, [see the docs](https://redis.io/docs/latest/operate/oss_and_stack/install/install-stack/)
3. With docker: `docker run -d --name redis-stack-server -p 6379:6379 redis/redis-stack-server:latest`

## Define the Redis Connection URL and Client

By default this notebook connects to the local instance of Redis Stack. **If you have your own Redis Enterprise instance** - replace REDIS_PASSWORD, REDIS_HOST and REDIS_PORT values with your own.

In [12]:
import os
from redis import Redis

# Replace values below with your own if using Redis Cloud instance
REDIS_HOST = os.getenv("REDIS_HOST", "localhost") # ex: "redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
REDIS_PORT = os.getenv("REDIS_PORT", "6379")      # ex: 18374
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")  # ex: "1TNxTEdYRDgIDKM2gDfasupCADXXXX"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

redis_client = Redis.from_url(REDIS_URL)

## Init CohereClient as LLM layer

In [2]:
from typing import Dict, List
import cohere
import os

class CohereClient():
    def __init__(self, api_key: str = None, model: str = 'command-r-plus'):
        api_key = api_key or os.getenv("COHERE_API_KEY")
        self.client = cohere.Client(api_key)
        self._model = model

    def converse(self, prompt: str, context: List[Dict]) -> str:
        context = self.remap(context)
        response = self.client.chat(
                model=self._model,
                chat_history = context,
                message=prompt,
                )
        return response.text

    def remap(self, context) -> List[Dict]:
        ''' re-index the chat history to match the Cohere API requirements '''
        new_context = []
        for statement in context:
            if statement["role"] == "user":
                new_statement = {"role": "USER", "message": statement["content"]}
            elif statement["role"] == "llm":
                new_statement = {"role": "CHATBOT", "message": statement["content"]}
            elif statement["role"] == "system":
                new_statement = {"role": "SYSTEM", "message": statement["content"]}
            else:
                raise ValueError(f'Unknown chat role {statement["role"]}')
            new_context.append(new_statement)
        return new_context

## Import SemanticSessionManager

redisvl provides the SemanticSessionManager for easy management of session state.

In [14]:
from redisvl.extensions.session_manager import SemanticSessionManager

user_session = SemanticSessionManager(name='llm_chef', session_tag='123', user_tag='abc', redis_client=redis_client)
user_session.add_message({"role":"system", "content":"You are a helpful chef, assisting people in making delicious meals"})

client = CohereClient()

On each call to the LLM we can first retrieve the recent history of the conversation. On the first call it will just be the preamble we set.

In [15]:
prompt = "can you give me some ideas for breakfast?"
context = user_session.get_recent()
response = client.converse(prompt=prompt, context=context)
user_session.store(prompt, response)
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  can you give me some ideas for breakfast?

LLM:  Sure! Here are a few breakfast ideas:

- Avocado toast: This is a quick and easy option that's both delicious and nutritious. Simply toast some bread, smash an avocado on top, and add a squeeze of lemon or lime juice and a pinch of salt and pepper to taste. You can also add additional toppings like tomato, feta cheese, or a fried egg.
- Oatmeal: Oatmeal is a hearty and healthy breakfast option that can be prepared in a variety of ways. You can cook it on the stovetop or in the microwave, and top it with fruits, nuts, seeds, or a drizzle of honey.
- Smoothie bowl: Blend together your favorite fruits, vegetables, and liquid base (such as juice, milk, or yogurt) to make a smoothie, and then pour it into a bowl. Top it with granola, nuts, seeds, or fresh fruit for a beautiful and tasty breakfast.
- Eggs: There are endless ways to prepare eggs for breakfast. You can scramble, fry, poach, or boil them, and serve them with toast, avocado

Let's continue this conversation and inspect the context we're passing to the LLM as we go.

In [6]:
prompt = "can you give me the recipe for those pancakes?"
context = user_session.get_recent()
response = client.converse(prompt=prompt, context=context)
user_session.store(prompt, response)
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  can you give me the recipe for those pancakes?

LLM:  Sure, here's a basic recipe for pancakes that serves about 4 people (about 12 small pancakes):

Ingredients:

 - 2 cups all-purpose flour
 - 2 teaspoons baking powder
 - 1/2 teaspoon salt
 - 2 large eggs
 - 2 cups milk
 - 2 tablespoons butter, melted
 - Additional butter for the pan
 - Syrup, fruit, or other desired toppings

Instructions:

1. In a large bowl, whisk together the flour, baking powder, and salt.

2. In a separate bowl, beat the eggs, and then whisk in the milk and melted butter.

3. Pour the wet ingredients into the dry ingredients and stir until just combined. Be careful not to overmix; the batter should still be a little lumpy.

4. Heat a griddle or large skillet over medium heat, and grease it with butter.

5. Pour about 1/4 cup of batter for each pancake onto the griddle. Cook until the edges look dry and there are small bubbles on the surface, about 2-3 minutes.

6. Flip the pancakes and cook until the oth

In [7]:
for ctx in context:
    print(ctx)

{'role': 'system', 'content': 'You are a helpful chef, assisting people in making delicious meals'}
{'role': 'user', 'content': 'can you give me some ideas for breakfast?'}
{'role': 'llm', 'content': "Sure! Here are a few breakfast ideas:\n\n- Avocado toast: This is a quick and easy option that's both delicious and nutritious. Simply toast some bread, smash an avocado on top, and add a squeeze of lemon or lime juice and a pinch of salt and pepper to taste. You can also add additional toppings like sliced tomato, feta cheese, or a fried egg.\n\n- Yogurt parfait: Layer some Greek yogurt with fresh fruit (such as berries, chopped mango, or bananas), granola, and a drizzle of honey for a tasty and healthy breakfast.\n\n- Oatmeal: Oatmeal is a warm and filling option that can be customized in many ways. Try cooking oats with milk or water, and then topping with chopped nuts or nut butter, cinnamon, fruit, or a drizzle of maple syrup.\n\n- Breakfast sandwich: You can make a breakfast sandwic

In [8]:
prompt = "I am vegetarian. Can you remove the eggs?"
context = user_session.get_recent()
response = client.converse(prompt=prompt, context=context)
user_session.store(prompt, response)
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  I am vegetarian. Can you remove the eggs?

LLM:  Absolutely! Here's an egg-free version of the pancake recipe:

Ingredients:

 - 2 cups all-purpose flour
 - 2 teaspoons baking powder
 - 1/2 teaspoon salt
 - 2 cups milk (dairy or non-dairy alternative)
 - 2 tablespoons butter, melted (or substitute with vegetable oil)
 - Additional butter or oil for the pan
 - Syrup, fruit, or other desired toppings

Instructions:

1. In a large bowl, whisk together the flour, baking powder, and salt.

2. In a separate bowl, whisk together the milk and melted butter (or oil).

3. Pour the wet ingredients into the dry ingredients and stir until just combined. Be careful not to overmix; the batter should still be a little lumpy.

4. Heat a griddle or large skillet over medium heat, and grease it with butter or oil.

5. Pour about 1/4 cup of batter for each pancake onto the griddle. Cook until the edges look dry and there are small bubbles on the surface, about 2-3 minutes.

6. Flip the pancakes and

In [9]:
for ctx in context:
    print(ctx)

{'role': 'system', 'content': 'You are a helpful chef, assisting people in making delicious meals'}
{'role': 'user', 'content': 'can you give me some ideas for breakfast?'}
{'role': 'llm', 'content': "Sure! Here are a few breakfast ideas:\n\n- Avocado toast: This is a quick and easy option that's both delicious and nutritious. Simply toast some bread, smash an avocado on top, and add a squeeze of lemon or lime juice and a pinch of salt and pepper to taste. You can also add additional toppings like sliced tomato, feta cheese, or a fried egg.\n\n- Yogurt parfait: Layer some Greek yogurt with fresh fruit (such as berries, chopped mango, or bananas), granola, and a drizzle of honey for a tasty and healthy breakfast.\n\n- Oatmeal: Oatmeal is a warm and filling option that can be customized in many ways. Try cooking oats with milk or water, and then topping with chopped nuts or nut butter, cinnamon, fruit, or a drizzle of maple syrup.\n\n- Breakfast sandwich: You can make a breakfast sandwic

In [10]:
prompt = "I am also vegan. Can you replace the butter too?"
context = user_session.get_recent()
response = client.converse(prompt=prompt, context=context)
user_session.store(prompt, response)
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  I am also vegan. Can you replace the butter too?

LLM:  Sure, for a vegan-friendly version of the pancake recipe, you can replace the butter with a vegan alternative. Here's the updated recipe:

Ingredients:

 - 2 cups all-purpose flour
 - 2 teaspoons baking powder
 - 1/2 teaspoon salt
 - 2 cups plant-based milk (such as soy, almond, or oat milk)
 - 2 tablespoons neutral-flavored oil (such as canola or vegetable oil)
 - Additional oil or vegan butter for the pan
 - Syrup, fruit, or other desired toppings

Instructions:

1. In a large bowl, whisk together the flour, baking powder, and salt.

2zji2. In a separate bowl, whisk together the plant-based milk and oil.

3. Pour the wet ingredients into the dry ingredients and stir until just combined. Be careful not to overmix; the batter should still be a little lumpy.

4. Heat a griddle or large skillet over medium heat, and grease it with a small amount of oil or vegan butter.

5. Pour about 1/4 cup of batter for each pancake onto th

In [11]:
for ctx in context:
    print(ctx)

{'role': 'llm', 'content': "Sure! Here are a few breakfast ideas:\n\n- Avocado toast: This is a quick and easy option that's both delicious and nutritious. Simply toast some bread, smash an avocado on top, and add a squeeze of lemon or lime juice and a pinch of salt and pepper to taste. You can also add additional toppings like sliced tomato, feta cheese, or a fried egg.\n\n- Yogurt parfait: Layer some Greek yogurt with fresh fruit (such as berries, chopped mango, or bananas), granola, and a drizzle of honey for a tasty and healthy breakfast.\n\n- Oatmeal: Oatmeal is a warm and filling option that can be customized in many ways. Try cooking oats with milk or water, and then topping with chopped nuts or nut butter, cinnamon, fruit, or a drizzle of maple syrup.\n\n- Breakfast sandwich: You can make a breakfast sandwich with eggs, cheese, and your choice of breakfast meat (such as bacon, sausage, or ham) on an English muffin, bagel, or toast.\n\n- Smoothies: Blend together your choice of 

This works, but our context keeps growing, increasing our LLM token count, which increases latency and cost.
Conversation histories can be truncated, but that can break quickly.

In [10]:
prompt = "I changed my mind. Can you give me the first recipe from your list?"
context = user_session.get_recent(top_k=5)
response = client.converse(prompt=prompt, context=context)
user_session.store(prompt, response)
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  I changed my mind. Can you give me the first recipe from your list?

LLM:  Sure! Here's the recipe for classic fluffy pancakes with eggs and butter:

**Ingredients:**

- 2 cups (250 grams) all-purpose flour
- 2 teaspoons baking powder
- 1 teaspoon salt
- 2 tablespoons sugar
- 2 cups milk
- 2 eggs
- 3 tablespoons melted butter, plus more for the pan
- 1 teaspoon vanilla extract (optional)

**Instructions:**

1. Whisk together the dry ingredients: In a large mixing bowl, combine the flour, baking powder, salt, and sugar. Whisk until the ingredients are well incorporated and there are no lumps.

2. Combine the wet ingredients: In a separate mixing bowl, whisk together the milk, eggs, melted butter, and vanilla extract (if using). Ensure the melted butter has cooled down slightly so it doesn't cook the eggs.

3. Mix dry and wet ingredients: Slowly pour the wet ingredients into the dry ingredients while continuously whisking. Mix until you have a smooth batter, but be careful not to 

In [11]:
for ctx in context:
    print(ctx)

{'role': 'llm', 'content': "Of course! Here is a classic recipe for fluffy and delicious pancakes:\n\n**Ingredients:**\n\n- 2 cups (250 grams) all-purpose flour\n- 2 teaspoons baking powder\n- 1 teaspoon salt\n- 2 tablespoons sugar\n- 2 cups milk\n- 2 eggs\n- 3 tablespoons melted butter, plus more for the pan\n- 1 teaspoon vanilla extract (optional)\n\n**Instructions:**\n\n1. Whisk together the dry ingredients: In a large mixing bowl, combine the flour, baking powder, salt, and sugar. Whisk until well blended.\n\n2. Combine the wet ingredients: In a separate mixing bowl, whisk together the milk, eggs, melted butter, and vanilla extract (if using). Make sure the melted butter has cooled down a bit so it doesn't cook the eggs.\n\n3. Mix dry and wet ingredients: Slowly pour the wet ingredients into the dry ingredients while continuously whisking until you have a smooth batter. Be careful not to overmix; a few small lumps are okay.\n\n4. Heat your griddle or pan: Place a nonstick pan or gr

The LLM has forgotten about the avocado toast it mentioned because that was too far back in it's conversation history!

## Semantic session memory

Rather than naively pass the most recent conversation we can use RedisVL to find only the semantically relevant sections of our conversation based on our new prompt. We can do this with the `fetch_relevant()` method.

In [12]:
prompt = "Can you give me the avocado one?"
user_session.set_distance_threshold(0.75)
context = user_session.get_relevant(prompt=prompt)
response = client.converse(prompt=prompt, context=context)
user_session.store(prompt, response)
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  Can you give me the avocado one?

LLM:  Sure! Here's the first breakfast idea, Avocado Toast, with a simple recipe and some optional toppings:

**Avocado Toast**

**Ingredients:**
- 1 ripe avocado
- 2 slices of bread (preferably a hearty variety like sourdough, rye, or multigrain)
- Extra virgin olive oil
- Lemon or lime juice
- Salt and pepper, to taste
- Optional toppings: sliced tomatoes, chopped onions, chili flakes, or a fried egg

**Instructions:**

1. Toast the bread to your desired level of doneness.
2. While the bread is toasting, prepare the avocado. Cut the avocado in half, remove the pit, and scoop out the flesh into a small bowl.
3. Using a fork, mash the avocado until it reaches your desired consistency. You can leave it chunkier or make it smoother, depending on your preference.
4. Add a drizzle of extra virgin olive oil and a squeeze of lemon or lime juice to the mashed avocado. This will enhance the flavor and prevent the avocado from turning brown too quickly. 

In [13]:
for ctx in context:
    print(ctx)

{'role': 'user', 'content': 'can you give me some ideas for breakfast?'}
{'role': 'llm', 'content': "Certainly! Here are some delicious breakfast ideas to start your day off right:\n\n**1. Avocado Toast:**\n   - Spread mashed avocado on a slice of toasted bread. Top it with a sprinkle of salt and pepper, a squeeze of lemon or lime juice, and a drizzle of extra virgin olive oil. You can also add sliced tomatoes, chopped onions, or a fried egg on top.\n\n**2. Smoothie Bowl:**\n   - Blend your favorite fruits (such as bananas, strawberries, blueberries, and mango) with a liquid base (like milk, yogurt, or juice) to create a thick and creamy smoothie. Pour it into a bowl and top it with granola, nuts, seeds, shredded coconut, or fresh fruit slices.\n\n**3. Pancakes or Waffles:**\n   - Whip up a batch of fluffy pancakes or waffles and serve them with a variety of toppings. Some options include fresh fruit, maple syrup, butter, whipped cream, chocolate chips, or a dusting of powdered sugar.\

Looking at the context we see that only the conversation section that had to do with avocados was retrieved. We don't need to know about pancakes to make avocado toast.

In [14]:
user_session.clear()